In [ ]:
import socket
import cv2
import mediapipe as mp
import json
import numpy as np

# Initialize MediaPipe
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

# Open webcam
cap = cv2.VideoCapture(0)

# Create UDP socket
sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
server_address = ('127.0.0.1', 5050)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Flip frame to mirror image
    frame = cv2.flip(frame, 1)

    # Original feed for the left side
    original = frame.copy()

    # Black image for the skeleton side
    skeleton = np.zeros_like(frame)

    # Convert to RGB
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(rgb)

    data = {}

    if results.pose_landmarks:
        landmarks = results.pose_landmarks.landmark
        key_indices = [0, 11, 12, 13, 14, 23, 24]  # Nose, shoulders, elbows, hips

        for idx in key_indices:
            lm = landmarks[idx]
            data[str(idx)] = {'x': lm.x, 'y': lm.y, 'z': lm.z}
            print(f"Joint {idx}: x={lm.x:.2f}, y={lm.y:.2f}, z={lm.z:.2f}")

        # Send JSON to Unity
        message = json.dumps(data).encode()
        sock.sendto(message, server_address)
        print("Data sent to Unity 🚀")

        # Draw pose landmarks on the black image
        mp_drawing.draw_landmarks(
            skeleton,
            results.pose_landmarks,
            mp_pose.POSE_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=4),
            mp_drawing.DrawingSpec(color=(255, 0, 0), thickness=2)
        )
    else:
        print("❌ Pose not detected")

    # Combine both views side-by-side
    combined = np.hstack((original, skeleton))

    # Show the result
    cv2.imshow("You (Left) vs Skeleton (Right)", combined)

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

# Cleanup
cap.release()
cv2.destroyAllWindows()
sock.close()


In [ ]:
import cv2
import mediapipe as mp
import numpy as np

# Load the anime face image (must be in same folder as this notebook)
face_img = cv2.imread('/media/snow/Dracaryz/dancingai/.ipynb_checkpoints/anime_face-checkpoint.png', cv2.IMREAD_UNCHANGED)

# Check if image is loaded
if face_img is None:
    print("❌ Error: Could not load 'anime_face.png'. Make sure it's in the same folder.")
    raise SystemExit

# Resize face image
face_img = cv2.resize(face_img, (100, 100))

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose()

# Open webcam
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(rgb)

    # Left view: original webcam
    left = frame.copy()

    # Right view: black background
    right = np.zeros_like(frame)

    if results.pose_landmarks:
        # Draw skeleton on right canvas
        mp_drawing.draw_landmarks(
            right,
            results.pose_landmarks,
            mp_pose.POSE_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=4, circle_radius=4),
            mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=4)
        )

        # Get the nose landmark (landmark index 0)
        landmarks = results.pose_landmarks.landmark
        h, w, _ = right.shape
        nose = landmarks[0]
        cx, cy = int(nose.x * w), int(nose.y * h)

        # Resize face image (already 100x100)
        face_h, face_w, _ = face_img.shape
        x1 = cx - face_w // 2
        y1 = cy - face_h // 2
        x2 = x1 + face_w
        y2 = y1 + face_h

        # Make sure face overlay is inside bounds
        if x1 >= 0 and y1 >= 0 and x2 <= w and y2 <= h:
            roi = right[y1:y2, x1:x2]
            face_rgb = face_img[:, :, :3]
            face_alpha = face_img[:, :, 3] / 255.0

            # Alpha blending face image over black background
            for c in range(3):
                roi[:, :, c] = roi[:, :, c] * (1 - face_alpha) + face_rgb[:, :, c] * face_alpha

            right[y1:y2, x1:x2] = roi

    # Combine left and right views
    combined = np.hstack((left, right))

    # Show side-by-side view
    cv2.imshow("🧍 You (Left) | 🎎 Anime Twin (Right)", combined)

    # Press 'q' to quit
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

# Release everything
cap.release()
cv2.destroyAllWindows()


In [ ]:
import cv2
import mediapipe as mp
import numpy as np

# Load your full anime body image (with transparency)
anime_img = cv2.imread('anime_body.png', cv2.IMREAD_UNCHANGED)

if anime_img is None:
    print("❌ Could not load 'anime_body.png'. Make sure it’s in the same folder.")
    raise SystemExit

# Resize your anime body image
anime_img = cv2.resize(anime_img, (200, 300))  # You can change size here

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

# Start webcam
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(rgb)

    # Left = original webcam
    left = frame.copy()

    # Right = empty black canvas
    right = np.zeros_like(frame)

    if results.pose_landmarks:
        landmarks = results.pose_landmarks.landmark
        h, w, _ = right.shape

        # Use hip center to place anime body
        left_hip = landmarks[23]
        right_hip = landmarks[24]

        # Average of left and right hip
        cx = int(((left_hip.x + right_hip.x) / 2) * w)
        cy = int(((left_hip.y + right_hip.y) / 2) * h)

        # Overlay anime image at hip center
        img_h, img_w, _ = anime_img.shape
        x1 = cx - img_w // 2
        y1 = cy - img_h // 2
        x2 = x1 + img_w
        y2 = y1 + img_h

        if x1 >= 0 and y1 >= 0 and x2 <= w and y2 <= h:
            roi = right[y1:y2, x1:x2]
            char_rgb = anime_img[:, :, :3]
            char_alpha = anime_img[:, :, 3] / 255.0

            for c in range(3):
                roi[:, :, c] = roi[:, :, c] * (1 - char_alpha) + char_rgb[:, :, c] * char_alpha

            right[y1:y2, x1:x2] = roi

    # Combine both views
    combined = np.hstack((left, right))
    cv2.imshow("You (Left) | Anime Avatar (Right)", combined)

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
